In [ ]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os

from amb_sdk.sdk import DarwinSdk

In [ ]:
# Set local path to files
path = '/Users/kmoore/amb/sets'

In [ ]:
# View data snippet
df = pd.read_csv(os.path.join(path,'pulsars.csv'))
df.head()

In [ ]:
ds.set_url('https://darwin-api.sparkcognition.com/v1/')

In [ ]:
# Register user - pick username & password
""""api_key = 'KEY'
password = 'password'
ds = DarwinSdk()
status, msg = ds.auth_login(password, api_key)
if not status:
    print(msg)
status, msg = ds.auth_register_user('user', 'password')
if not status:
    print(msg)""""""

In [ ]:
# Login - put in the registered username & password
ds = DarwinSdk()
status, msg = ds.auth_login_user('user','password')
if not status:
    print(msg)

In [ ]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(path,'pulsars.csv'))
if not status:
    print(dataset)

In [ ]:
# Build model
model = "unsupervised_model0"
status, job_id = ds.create_model(dataset_names = 'pulsars.csv', \
                                 model_name =  model, \
                                 max_time = '00:05',
                                 n_clusters = 2)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
# Train some more
status, job_id = ds.resume_training_model(model, \
                                          'pulsars.csv', \
                                          max_train_time='0:05', \
                                          n_clusters=2, \
                                          supervised=False)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
# Test model
status, artifact = ds.run_model('pulsars.csv', model, supervised=False)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
# Get predictions
status, pred_file = ds.download_artifact(artifact['artifact_name'])
prediction = pd.read_csv(pred_file['filename'])

In [ ]:
prediction.head()

In [ ]:
#Plot predictions vs actual
prediction['prediction'].plot(title='Clusters', marker='.', linewidth=0)
plt.xlabel('sample')
plt.figure()
prediction['anomaly'].plot(title='Anomaly Score')


In [ ]:
# Plot couple of features and color them with cluter number
plt.scatter(df['mean_profile'],df['std_profile'],c=prediction['prediction'])

In [ ]:
#Comparing labels to clusters from unsupervised
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(df['class'], prediction['prediction'])
print(cm)
print(classification_report(df['class'], prediction['prediction']))

In [ ]:
# Delete all models and datasets
ds.delete_all_datasets()
ds.delete_all_models()